In [1]:
with open("../data/day9.txt") as f:
    data = f.read()

In [2]:
def get_expanded_map(map):
    result = []
    for i, length in enumerate(map):
        length = int(length)
        if (i % 2) == 0:
            result.extend([i // 2] * length)
        else:
            result.extend("." * length)
    return result


expanded_map = get_expanded_map(data)

In [3]:
def find_last_number_and_index(lst, last_index=None):
    if last_index < 0:
        last_index += len(lst)
    for index in range(last_index, -1, -1):

        if isinstance(lst[index], int):

            return lst[index], index

    return None, -1



def find_first_free_spot(map, starting_index=0):
    return map[starting_index:].index(".") + starting_index



def swap_last_number_with_first_free_space(expanded_map, first_index=0, last_index=-1):
    # Keeping track of the last found indices of a number and a "." speeds things up massively

    first_free_space = find_first_free_spot(expanded_map, first_index)

    last_number, last_index = find_last_number_and_index(
        expanded_map, last_index=last_index
    )

    if last_index < first_free_space:

        raise ValueError("Done")

    expanded_map[first_free_space] = last_number

    expanded_map[last_index] = "."

    return expanded_map, first_free_space, last_index



def main():

    map = expanded_map.copy()
    first_index, last_index = 0, -1

    while True:

        try:

            map, first_index, last_index = swap_last_number_with_first_free_space(
                map, first_index, last_index
            )

        except ValueError:
            return map

In [ ]:
def calculate_checksum(map):
    result = 0
    for i, id_ in enumerate(map):
        if id_ != ".":
            result += i * id_
    return result


map = main()
calculate_checksum(map)

# Part 2

In [5]:
# My actuall submission was this brute force, which takes ~2 min
#
# block_lengths = [(i, int(length)) for i, length in enumerate(data[::2])]

# defragmented_map = expanded_map.copy()



# def find_first_free_spot(map, length):

#     for i in range(len(map) - length - 1):

#         if map[i : i + length] == ["."] * length:

#             return i

#     raise ValueError("No placement possible")



# for id, length in tqdm.tqdm(block_lengths[::-1]):  # Try only once

#     original_index = expanded_map.index(id)

#     try:

#         new_index = find_first_free_spot(defragmented_map, length)

#     except ValueError:

#         continue

#     if new_index > original_index:

#         continue

#     defragmented_map[new_index : new_index + length] = [id] * length
#     defragmented_map[original_index : original_index + length] = ["."] * length

## Performance improvement
This can be done analytically

In [6]:
import numpy as np

In [ ]:
cumsum = np.cumsum([0] + [int(x) for x in data])
free_spaces = [(index, int(space)) for index, space in zip(cumsum[1::2], data[1::2])]
block_lengths = [
    (i, index, int(length))
    for i, (index, length) in enumerate(zip(cumsum[::2], data[::2]))
]

defragmented_map = expanded_map.copy()
for id, current_ix, length in block_lengths[::-1]:  # Try only once
    for i, (possible_new_ix, space) in enumerate(free_spaces):
        if current_ix < possible_new_ix:
            break  # could not place
        if space >= length:
            free_spaces.pop(i)
            defragmented_map[possible_new_ix : possible_new_ix + length] = [id] * length
            defragmented_map[current_ix : current_ix + length] = ["."] * length

            remaining_space = space - length
            if remaining_space:
                free_spaces.insert(i, (possible_new_ix + length, remaining_space))
            break

calculate_checksum(defragmented_map)